Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Xin Qu"
COLLABORATORS = ""

---

## A few points:
* Refer to the Datalog Assignment before attempting this notebook.
* It's important to run following cell first for rest of notebook to work.
* It's always a good idea to run cells in order. In case you have run cells in jumbled order and would want to start fresh, restart kernel from menu above.
* All clingo cells start with `%%clingo`.
* You can run your clingo cell against some basic facts and rules from a file. `set_db_file $filepath` sets the file against which your clingo cells will run.
* Each clingo cell is independent of others. Rules defined in one cell won't be available in others.
* It's nice to be able to execute clingo from within your notebook but don't forget to practice from command line. `%%clingo` is just a thin wrapper over command line and it's best to know how to use the underlying tool.
* Upon assignment submission, we will run your code against different set of facts. Please don't hardcode answers and save yourself the embarassment.

### Good luck!!

# In this problem we will use family dataset and write some datalog rules.
In this assignment you will be using Datalog as a query language for provenance. Provenance graphs are directed, acyclic graphs (DAGs), connecting nodes (e.g., data elements, workflow steps, etc.) through directed edges. Edges typically model dataflow, e.g., read and write events in the case of retrospective provenance information, or in and out edges in a prospective provenance graph (i.e., a workflow graph).

Similarly, provenance graphs might also represent the “backward dependencies” (used, was generated by) of the Open Provenance Model (OPM) or the W3C PROV model.

Problem 1 resumes the “family relations” theme from an earlier assignment, using relationships that are equally useful when querying real data provenance graphs. For example, a recursively defined relation such as anc(x,y) (for ancestor) can be used to trace the lineage of a data product from a workflow run. 

![Ancestor](ancestor.png "Ancestor")

In G there is an edge from x to y, if x is a child of y, or equivalently, if x has a parent y. In the Datalog program P this is represented by a fact par(x,y). The rules for anc(x,y) define the transitive ancestor relation, i.e., anc(x,y) is true iff one can reach from x an ancestor y via a chain of parent edges. Similarly, ca(x,y,a) means that a is a common ancestor of x and y, and lca(x,y,a) means that a is the least common ancestor (LCA) of x and y. To compute the LCA, we use an auxiliary relation not lca(x,y,a), which states that a is not the LCA of x and y (since there exists a “lower” common ancestor a1).

In [2]:
%reload_ext lib.clingo.clingo_magic
import os
from lib.clingo.clingo_evaluate_util import clingo_evaluate

In [3]:
# All clingo cells will run against this file containing some base facts.
family_base_facts_and_rules_file = 'problem1-lca.lp'
%set_db_file $family_base_facts_and_rules_file

'% The parent facts par(Child, Parent)\npar(albert,bernhard). \npar(albert,monique).\npar(bernhard,hermann). \npar(bernhard,johanna).\npar(eva,kati).\npar(franz,rita). \npar(franz,wilhelm).  \npar(heinz,rita). \npar(heinz,wilhelm).\npar(hermann,rita).\npar(hermann,wilhelm). \npar(karl,hermann). \npar(karl,johanna). \npar(kati,hermann).\npar(kati,johanna).'

### [5 points] 1. ancestor Albert
Provide a datalog rule for finding ancestors of Albert

In [4]:
%%clingo {"predicate" : "ancestor_albert", "predicate_arity" : 1, "result_var": "Ancestor_albert"}

% Change following expression.
anc(X,Y) :- par(X,Y).
anc(X,Y) :- par(X,Z), anc(Z,Y).

ancestor_albert(X) :- anc(albert, X).



Saving output to local variable Ancestor_albert['result']
Saving code snippet to local variable Ancestor_albert['code']


'ancestor_albert(bernhard) ancestor_albert(monique) ancestor_albert(johanna) ancestor_albert(hermann) ancestor_albert(rita) ancestor_albert(wilhelm)'


#### Test 1 for ancestor albert.
Following test will compare output of your ancestor_albert rule against expected output.
You must have run all clingo cells above for test to pass.

In [5]:
# Following should be output of your previous cell.
# Order of predicates in the output doesn't matter.
# Run to see expected output with syntax highlighting.
expected_output = '''ancestor_albert(bernhard) ancestor_albert(monique) ancestor_albert(johanna) ancestor_albert(hermann) ancestor_albert(rita) ancestor_albert(wilhelm)'''
db_file = 'problem1-lca.lp'
clingo_evaluate(db_file, Ancestor_albert['code'], 'ancestor_albert', 1, expected_output)

'ancestor_albert(bernhard) ancestor_albert(monique) ancestor_albert(johanna) ancestor_albert(hermann) ancestor_albert(rita) ancestor_albert(wilhelm)'

In [6]:
# Hidden Test
# Omitted from student's version of notebook.




### [5 points] 2. ancestor Eva.
Provide a datalog rule for finding ancestors of Eva

In [7]:
%%clingo {"predicate" : "ancestor_eva", "predicate_arity" : 1, "result_var": "Ancestor_eva"}

% Change following expression.
anc(X,Y) :- par(X,Y).
anc(X,Y) :- par(X,Z), anc(Z,Y).

ancestor_eva(X) :- anc(eva, X).



Saving output to local variable Ancestor_eva['result']
Saving code snippet to local variable Ancestor_eva['code']


'ancestor_eva(kati) ancestor_eva(johanna) ancestor_eva(hermann) ancestor_eva(rita) ancestor_eva(wilhelm)'

#### Test 2 for ancestor eva.
Following test will compare output of your ancestor_eva rule against expected output.
You must have run all clingo cells above for test to pass.

In [8]:
# Following should be output of your previous cell.
# Order of predicates in the output doesn't matter.
# Run to see expected output with syntax highlighting.
expected_output = '''ancestor_eva(kati) ancestor_eva(johanna) ancestor_eva(hermann) ancestor_eva(rita) ancestor_eva(wilhelm)'''
db_file = 'problem1-lca.lp'
clingo_evaluate(db_file, Ancestor_eva['code'], 'ancestor_eva', 1, expected_output)

'ancestor_eva(kati) ancestor_eva(johanna) ancestor_eva(hermann) ancestor_eva(rita) ancestor_eva(wilhelm)'

In [9]:
# Hidden Test
# Omitted from student's version of notebook.




### [6 points] 3. Common Ancestors of Albert and Eva
Provide a datalog rule for finding the common ancestors of Albert and Eva

In [10]:
%%clingo {"predicate" : "ca_albert_eva", "predicate_arity" : 1, "result_var": "Ca_albert_eva"}

% Change following expression.
anc(X,Y) :- par(X,Y).
anc(X,Y) :- par(X,Z), anc(Z,Y).

ca(X,A,A) :- anc(X,A).
ca(A,X,A) :- anc(X,A).
ca(X,Y,A) :- anc(X,A), anc(Y,A), X != Y. 

ca_albert_eva(X) :- ca(albert, eva, X).



Saving output to local variable Ca_albert_eva['result']
Saving code snippet to local variable Ca_albert_eva['code']


'ca_albert_eva(johanna) ca_albert_eva(hermann) ca_albert_eva(rita) ca_albert_eva(wilhelm)'

#### Test 3 for common Ancestors of Albert and Eva.
Following test will compare output of your ca_albert_eva rule against expected output.
You must have run all clingo cells above for test to pass.

In [11]:
# This cell will test the descendant with these new facts.
# Contents of this cell will not be present in student's version of assignment.
# This will only be evaluated after submission.

expected_output = '''ca_albert_eva(johanna) ca_albert_eva(hermann) ca_albert_eva(rita) ca_albert_eva(wilhelm)'''

db_file = 'problem1-lca.lp'
clingo_evaluate(db_file, Ca_albert_eva['code'], 'ca_albert_eva', 1, expected_output)


'ca_albert_eva(johanna) ca_albert_eva(hermann) ca_albert_eva(rita) ca_albert_eva(wilhelm)'

In [12]:
# Hidden Test
# Omitted from student's version of notebook.


### [12 points] 4. Which of the common ancestors in (3) are LCAs
Provide datalog rules for the lowest common ancestors for Albert and Eva

In [13]:
%%clingo {"predicate" : "lca_albert_eva", "predicate_arity" : 1, "result_var": "Lca_albert_eva"}

% Change following expression.
anc(X,Y) :- par(X,Y).
anc(X,Y) :- par(X,Z), anc(Z,Y).

ca(X,A,A) :- anc(X,A).
ca(A,X,A) :- anc(X,A).
ca(X,Y,A) :- anc(X,A), anc(Y,A), X != Y.  

% provide negative rules for not lca
not_lca(X,Y,A) :- ca(X,Y,A), ca(X,Y,A1), anc(A1,A).

lca(X,Y,A) :- ca(X,Y,A), not not_lca(X,Y,A).

lca_albert_eva(X) :- lca(albert,eva,X).



Saving output to local variable Lca_albert_eva['result']
Saving code snippet to local variable Lca_albert_eva['code']


'lca_albert_eva(johanna) lca_albert_eva(hermann)'


#### Test 4 for LCA
Following test will compare output of your lca_albert_eva rule against expected output.
You must have run all clingo cells above for test to pass.

In [14]:
expected_output = '''
lca_albert_eva(johanna) lca_albert_eva(hermann)
'''
db_file = 'problem1-lca.lp'
clingo_evaluate(db_file, Lca_albert_eva['code'], 'lca_albert_eva', 1, expected_output)

'lca_albert_eva(johanna) lca_albert_eva(hermann)'

### [12 points] 5. Which of the common ancestors in (3) are not the LCAs
Provide datalog rules for the common ancestor of Albert and Eva that are not the LCA

In [15]:
%%clingo {"predicate" : "nlca_albert_eva", "predicate_arity" : 1, "result_var": "Nlca_albert_eva"}

% Change following expression.
anc(X,Y) :- par(X,Y).
anc(X,Y) :- par(X,Z), anc(Z,Y).

ca(X,A,A) :- anc(X,A).
ca(A,X,A) :- anc(X,A).
ca(X,Y,A) :- anc(X,A), anc(Y,A), X != Y.  


% provide negative rules for not lca
not_lca(X,Y,A) :- ca(X,Y,A), ca(X,Y,A1), anc(A1,A).

lca(X,Y,A) :- ca(X,Y,A), not not_lca(X,Y,A).

nlca_albert_eva(X) :- not_lca(albert,eva,X).




Saving output to local variable Nlca_albert_eva['result']
Saving code snippet to local variable Nlca_albert_eva['code']


'nlca_albert_eva(rita) nlca_albert_eva(wilhelm)'


#### Test 5 for not LCA
Following test will compare output of your nlca_albert_eva rule against expected output.
You must have run all clingo cells above for test to pass.

In [16]:
expected_output = '''
nlca_albert_eva(rita) nlca_albert_eva(wilhelm)
'''
db_file = 'problem1-lca.lp'
clingo_evaluate(db_file, Nlca_albert_eva['code'], 'nlca_albert_eva', 1, expected_output)

'nlca_albert_eva(rita) nlca_albert_eva(wilhelm)'